In [ ]:
from glob import glob
import pickle
import openai
import os
from tqdm import tqdm
import time

openai.api_key = ''

In [2]:
ques_path = '../../data/ques_fixed/'
saves = glob(f'{ques_path}/*.pkl')
print(len(saves))

resp_path = 'resp'

context = """\
You are a very good translator. \
The text I want to translate is the Korean pharmacist national exam. \
I type in the Korean questions and you convert them to English in their original form.\
"""

20


In [3]:
from concurrent.futures import ThreadPoolExecutor

def process_and_update(ques, content, pbar):
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": context},
                    {"role": "user", "content": f"{ques}"}
                ]
            )
            pbar.update(1)
            return response.choices[0].message.content
        except Exception as e:
            retries += 1
            print(e)
            print(f"Retrying... ({retries}/{max_retries})")
            time.sleep(60)

def process_exam(path):
    with open(path, 'rb') as f:
        exam = pickle.load(f)
    f.close()
    filename = os.path.basename(path)
    resp = []
    with tqdm(total=len(exam),desc=filename) as pbar:
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(lambda ques: process_and_update(ques, context, pbar), exam))
        
        for result in results:
            resp.append(result)

    if not os.path.exists(resp_path):
        os.makedirs(resp_path)
    with open(f'{resp_path}/{filename}', 'wb') as f:
        pickle.dump(resp, f)

for path in saves:
    process_exam(path)

2019_70_1.pkl:   0%|          | 0/100 [00:00<?, ?it/s]

2020_71_2.pkl:  42%|████▏     | 38/90 [00:30<00:38,  1.34it/s]

The server is overloaded or not ready yet.
Retrying... (1/5)


2023_74_4.pkl: 100%|██████████| 83/83 [01:18<00:00,  1.05it/s]
